In [ ]:
import numpy as np
import pandas as pd
import cv2
from skimage.feature import graycomatrix, graycoprops
from skimage.transform import resize
from skimage.color import rgb2gray
import os
from tqdm.notebook import tqdm
from skimage.io import imread

# Path dataset (sesuaikan dengan path Anda)
dataset_path = '/content/drive/MyDrive/Pemrograman data mining/P6/Pistachio_Image_Dataset'

In [ ]:
def load_and_preprocess(image_path):
    """Load image dan preprocess ke grayscale 224x224"""
    img = imread(image_path)
    if img.ndim == 3:  # Jika RGB
        img = rgb2gray(img)
    img_resized = resize(img, (224, 224), anti_aliasing=True)
    img_uint8 = (img_resized * 255).astype(np.uint8)
    return img_uint8

def extract_glcm_features(img_gray):
    """Ekstrak fitur GLCM dari gambar grayscale"""
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0°, 45°, 90°, 135°
    distances = [1]

    glcm = graycomatrix(img_gray, distances=distances, angles=angles,
                         symmetric=True, normed=True)

    features = {}
    for angle_idx, angle in enumerate(["0", "45", "90", "135"]):
        features[f'contrast_{angle}'] = graycoprops(glcm, 'contrast')[0, angle_idx]
        features[f'dissimilarity_{angle}'] = graycoprops(glcm, 'dissimilarity')[0, angle_idx]
        features[f'homogeneity_{angle}'] = graycoprops(glcm, 'homogeneity')[0, angle_idx]
        features[f'energy_{angle}'] = graycoprops(glcm, 'energy')[0, angle_idx]
        features[f'correlation_{angle}'] = graycoprops(glcm, 'correlation')[0, angle_idx]
        features[f'ASM_{angle}'] = graycoprops(glcm, 'ASM')[0, angle_idx]

    return features

# Main execution
results = []
for root, dirs, files in os.walk(dataset_path):
    for file in tqdm(files):
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            path = os.path.join(root, file)
            try:
                img = load_and_preprocess(path)
                feats = extract_glcm_features(img)
                feats['filename'] = os.path.basename(path)
                # Tambahkan label kelas berdasarkan nama folder
                class_name = os.path.basename(root)
                feats['class'] = class_name
                results.append(feats)
            except Exception as e:
                print(f"Error di {file}: {e}")

# Simpan ke CSV
if results:
    df = pd.DataFrame(results)
    output_path = '/content/drive/MyDrive/Pemrograman data mining/P6/hasil_glcm.csv'
    df.to_csv(output_path, index=False)
    print(f"Hasil disimpan di {output_path}")
    print(f"Total gambar yang diproses: {len(results)}")
else:
    print("Tidak ada hasil yang diperoleh. Cek path dataset Anda.")